In [1]:
pip install face-recognition

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install Dlib

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install chart_studio

Note: you may need to restart the kernel to use updated packages.


In [4]:
import face_recognition
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Face_to_BMI/Data/annotation.csv')

In [ ]:
df

In [ ]:
dff = df[0:513]
dfm = df[513:]

In [ ]:
dff.head()

In [ ]:
dfm.head()

**Females**

In [ ]:
sns.set()
f_plot = dff.plot(style='o', x='height', y='weight', figsize=(15,10), grid=True, markersize=10, legend=False, fontsize=12)
f_plot.set_ylabel('Weight(kg)', fontsize=12)
f_plot.set_xlabel('Height(m)', fontsize=12)

In [ ]:
print('Average Height = ' + str(round(dff['height'].mean(), 2)) + 'm, Average Weight = ' + str(round(dff['weight'].mean(), 2)) + 'kg')

**Males**

In [ ]:
f_plot = dfm.plot(style='o', x='height', y='weight', figsize=(15,10), grid=True, markersize=10, legend=False, fontsize=12)
f_plot.set_ylabel('Weight(kg)', fontsize=12)
f_plot.set_xlabel('Height(m)', fontsize=12)

In [ ]:
print('Average Height = ' + str(round(dfm['height'].mean(), 2)) + 'm, Average Weight = ' + str(round(dfm['weight'].mean(), 2)) + 'kg')

# Extract face embedding using facenet pretrained architecture

In [ ]:
def get_face_encoding(image_path):
    try:
      picture_of_me = face_recognition.load_image_file(image_path)
    except FileNotFoundError:
      print(image_path + " no file found")
      return np.zeros(128).tolist()
    my_face_encoding = face_recognition.face_encodings(picture_of_me)
    if not my_face_encoding:
        print(image_path + " no face found !!!")
        return np.zeros(128).tolist()
    return my_face_encoding[0].tolist()

In [6]:
all_faces = []
for image_name in df.image:
  face_enc = get_face_encoding('/content/drive/My Drive/Face_to_BMI/Data/Images_vip/'+image_name+'.jpg')
  all_faces.append(face_enc)

NameError: name 'df' is not defined

In [7]:
X = np.array(all_faces) ## This is the training data matrix
y_height = df.height.values ## all labels
y_weight = df.weight.values
y_BMI = df.BMI.values
plotX = pd.DataFrame(X)

NameError: name 'df' is not defined

**Visualisation of Face Embeddings by dimension reduction using PCA with respect to height and weight**

*PCA is an algorithm that is used for dimensionality reduction - meaning, informally, that it can take in a DataFrame with many columns and return a DataFrame with a reduced number of columns that still retains much of the information from the columns of the original DataFrame.* 

In [8]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(plotX)
print(pca.explained_variance_ratio_)
principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])
pca1 = principalDf['principal component 1'].tolist()
pca2 = principalDf['principal component 2'].tolist()
sns.set()
#plt.figure(figsize=(15,10))
fig = plt.figure(figsize=(15,10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(y_height, pca1, pca2, s=60)
ax.set_xlabel('Height(m)', fontsize=12)
ax.set_ylabel('PCA 1', fontsize=12)
ax.set_zlabel('PCA 2', fontsize=12)

NameError: name 'plotX' is not defined

In [ ]:
fig = plt.figure(figsize=(15,10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(y_weight, pca1, pca2, s=60)
ax.set_xlabel('Weight(kg)', fontsize=12)
ax.set_ylabel('PCA 1', fontsize=12)
ax.set_zlabel('PCA 2', fontsize=12)
plt.show()

# Splitting the dataset

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_height_train, y_height_test, y_weight_train, y_weight_test ,y_BMI_train, y_BMI_test = train_test_split(X, y_height,y_weight,y_BMI, random_state=1)

# Metric to check the goodness of fit

*Logarithmic transformation is a convenient means of transforming a highly skewed variable into a more normalized dataset.*  
*We want to produce the smallest error possible when making a prediction, while also taking into account that we should not be overfitting the model.Using the logarithm of one or more variables improves the fit of the model by transforming the distribution of the features to a more normally-shaped bell curve.*

In [ ]:
def report_goodness(model, X_test, y_test, predictor_log=True):
    # Make predictions using the testing set
    y_pred = model.predict(X_test)
    y_true = y_test

    if predictor_log:
        y_true = np.log(y_test)
    # The coefficients
    # The mean squared error
    print("Mean squared error: %.2f"      % mean_squared_error(y_true, y_pred))
    # Explained variance score: 1 is perfect prediction
    print('Variance score: %.2f' % r2_score(y_true, y_pred))
    
    errors = abs(y_pred - y_true)
    mape = 100 * np.mean(errors / y_true)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))

# Model selection
We will compare few regression model and select the one with better test score.

We will compare :

**Linear Regression**

**Ridge Linear Regression**

**Random Forest Regressor**

**Kernel Ridge Regression**

In [9]:
from sklearn.kernel_ridge import KernelRidge
from sklearn import  linear_model
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

**Simple Linear Regression**

*Linear regression, also known as ordinary least squares (OLS) and linear least squares, is the real workhorse of the regression world. Use linear regression to understand the mean change in a dependent variable given a one-unit change in each independent variable.*

**Height**

In [10]:
model_height = linear_model.LinearRegression()
model_height = model_height.fit(X_train, np.log(y_height_train))
report_goodness(model_height, X_test, y_height_test)

NameError: name 'X_train' is not defined

**Weight**

In [11]:
model_weight = linear_model.LinearRegression()
model_weight = model_weight.fit(X_train, np.log(y_weight_train))
report_goodness(model_height, X_test, y_weight_test)

NameError: name 'X_train' is not defined

*We can see we have got a negative value of R2. This means the the distnace between the predicted and actual height is very high or in another words our model has underfitted.*

**BMI**

In [12]:
model_BMI = linear_model.LinearRegression()
model_BMI = model_BMI.fit(X_train, np.log(y_BMI_train))
report_goodness(model_height, X_test, y_BMI_test)

NameError: name 'X_train' is not defined

*We have similar performance in case of BMI as well*

**Ridge Linear Regression**

*Ridge regression allows you to analyze data even when severe multicollinearity is present and helps prevent overfitting. This type of model reduces the large, problematic variance that multicollinearity causes by introducing a slight bias in the estimates. The procedure trades away much of the variance in exchange for a little bias, which produces more useful coefficient estimates when multicollinearity is present.*

**Height**

In [13]:
model_height = Ridge(fit_intercept=True, alpha=0.0015, random_state=4, normalize=True)
model_height = model_height.fit(X_train, np.log(y_height_train))
report_goodness(model_height, X_test, y_height_test)

NameError: name 'X_train' is not defined

**Weight**

In [14]:
model_weight = Ridge(fit_intercept=True, alpha=0.0015, random_state=4, normalize=True)
model_weight = model_weight.fit(X_train, np.log(y_weight_train))
report_goodness(model_weight, X_test, y_weight_test)

NameError: name 'X_train' is not defined

**BMI**

In [15]:
model_BMI = Ridge(fit_intercept=True, alpha=0.0015, random_state=4, normalize=True)
model_BMI = model_BMI.fit(X_train, np.log(y_BMI_train))
report_goodness(model_BMI, X_test, y_BMI_test)

NameError: name 'X_train' is not defined

**Random Forest Regressor**

**Height**

In [16]:
model_height = RandomForestRegressor(max_depth=2, random_state=0, n_estimators=100)
model_height = model_height.fit(X_train,np.log(y_height_train))
report_goodness(model_height, X_test, y_height_test)

NameError: name 'X_train' is not defined

**With hyperparameter tuning**

In [17]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 1000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 100, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rf = RandomForestRegressor()

In [18]:
rf_height_model = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)
# Fit the random search model
rf_height_model.fit(X_train,np.log(y_height_train))
report_goodness(rf_height_model, X_test, y_height_test)

NameError: name 'X_train' is not defined

**Weight**

In [19]:
model_weight = RandomForestRegressor(max_depth=2, random_state=0, n_estimators=100)
model_weight = model_weight.fit(X_train, np.log(y_weight_train))
report_goodness(model_weight, X_test, y_weight_test)

NameError: name 'X_train' is not defined

**With hyperparameter tuning**

In [20]:
rf_weight_model = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)
# Fit the random search model
rf_weight_model.fit(X_train,np.log(y_weight_train))
report_goodness(rf_weight_model,X_test,y_weight_test)

NameError: name 'X_train' is not defined

**BMI**

In [21]:
model_BMI = RandomForestRegressor(max_depth=2, random_state=0, n_estimators=100)
model_BMI = model_BMI.fit(X_train, np.log(y_BMI_train))
report_goodness(model_BMI, X_test, y_BMI_test)

NameError: name 'X_train' is not defined

**With hyperparameter tuning**

In [22]:
rf_BMI_model = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)
# Fit the random search model
rf_BMI_model.fit(X_train,np.log(y_BMI_train))
report_goodness(rf_BMI_model, X_test, y_BMI_test)

NameError: name 'X_train' is not defined

**Kernel Ridge Regression**

**Height**

In [23]:
model_height_kridge = KernelRidge(kernel='rbf', gamma=0.21, alpha=0.0017)
model_height_kridge = model_height_kridge.fit(X_train, np.log(y_height_train))
report_goodness(model_height_kridge, X_test, y_height_test)

NameError: name 'X_train' is not defined

**Weight**

In [24]:
model_weight_kridge = KernelRidge(kernel='rbf', gamma=0.21, alpha=0.0017)
model_weight_kridge = model_weight_kridge.fit(X_train, np.log(y_weight_train))
report_goodness(model_weight_kridge, X_test, y_weight_test)

NameError: name 'X_train' is not defined

**BMI**

In [25]:
model_BMI_kridge = KernelRidge(kernel='rbf', gamma=0.21, alpha=0.0017)
model_BMI_kridge = model_BMI_kridge.fit(X_train, np.log(y_BMI_train))
report_goodness(model_BMI_kridge, X_test, y_BMI_test)

NameError: name 'X_train' is not defined

**Conclusion :**

*We found that Random Forest Regresssor with hyperparameter tuning outperformed all the models in terms of the mean squared error and expalined variance*

**Save All Models**

In [26]:
from sklearn.externals import joblib

height_model = '/content/drive/My Drive/Face_to_BMI/Models/rf_height_model'
weight_model = '/content/drive/My Drive/Face_to_BMI/Models/rf_weight_model'
bmi_model = '/content/drive/My Drive/Face_to_BMI/Models/rf_BMI_model'

ImportError: cannot import name 'joblib' from 'sklearn.externals' (C:\Anaconda\lib\site-packages\sklearn\externals\__init__.py)

In [27]:
joblib.dump(rf_height_model, height_model)
joblib.dump(rf_weight_model, weight_model)
joblib.dump(rf_BMI_model, bmi_model)

NameError: name 'joblib' is not defined

**Load the Models**

In [28]:
height_predictor = joblib.load(height_model)
weight_predictor = joblib.load(weight_model)
bmi_predictor = joblib.load(bmi_model)

NameError: name 'joblib' is not defined

**Predict**

In [29]:
def predict_height_width_BMI(test_image, height_model, weight_model, bmi_model):
    test_array = np.expand_dims(np.array(get_face_encoding(test_image)), axis=0)
    height = np.exp(height_model.predict(test_array)).item()
    weight = np.exp(weight_model.predict(test_array)).item()
    bmi = np.exp(bmi_model.predict(test_array)).item()
    return 'Height: {}m\nWeight: {}kg\nBmi: {}kg/m^2'.format(round(height, 2), round(weight, 2), round(bmi, 2))

In [30]:
#@title
import ipywidgets as widgets
from IPython.display import display 
import os
images = os.listdir('/content/drive/My Drive/Face_to_BMI/Test Images')
for img in images:
  img_loc = '/content/drive/My Drive/Face_to_BMI/Test Images/' + img
  img_file = open(img_loc, 'rb')
  test_img = img_file.read()
  pred = predict_height_width_BMI(img_loc, height_predictor, weight_predictor, bmi_predictor)
  display(widgets.Image(value=test_img, format='jpg', width=300, height=400))
  print(pred)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/drive/My Drive/Face_to_BMI/Test Images'